#체지방률 예측
나이, 체중, 신장 등 관련 데이터로부터 체지방률을 예측하는, 회귀 문제를 해결해 봅시다.

본 과제는 2점 만점이며, 결정계수(Coefficient of Determination)를 평가 지표로 사용합니다.

Baseline보다 높은 결정계수 달성: 2.0점

Baseline의 결정계수와 동일(소수 5 번째 자리까지)하거나, 낮은 결정계수 달성: 1.0점

미제출: 0점

(주의) 허용할 수 없을 정도로 낮은 결정계수로 기록된 것도 미제출로 간주될 수 있습니다.

현재 Baseline은 scikit-learn 라이브러리의 KNeighborsRegressor 클래스를 활용해 도출됐습니다.

데이터 세트 분할 시, random_state 값은 42로 지정했습니다.

수업 시간에 다뤘던 k-Nearest Neighbors 알고리즘 및 하이퍼파라미터와 관련된 내용을 복습하고, 과제 2를 해결해 봅시다.

In [2]:
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

# 1. 데이터 불러오기
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("submission.csv")

# 2. 특징(X), 타깃(y) 분리
X = train.drop(columns=["BodyFat"])
y = train["BodyFat"]

# 3. 성능 확인을 위한 train/valid 분할 (random_state=42 필수!)
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. KNN + 스케일링 파이프라인
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

# 5. 하이퍼파라미터 그리드 설정
param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11, 15],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2],  # 1: 맨해튼, 2: 유클리드
}

# 6. GridSearchCV로 최적 하이퍼파라미터 탐색
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,           # 5-fold 교차검증
    scoring="r2",
    n_jobs=-1       # 가능하면 전체 코어 사용
)

grid.fit(X_tr, y_tr)

print("best params:", grid.best_params_)
print("cv best score (R^2):", grid.best_score_)

# 7. 검증 셋에서 성능 확인
val_pred = grid.predict(X_val)
val_r2 = r2_score(y_val, val_pred)
print("validation R^2:", val_r2)

# 8. 최종 모델을 전체 train 데이터로 다시 학습
best_model = grid.best_estimator_
best_model.fit(X, y)

# 9. test 데이터에 대한 예측 수행
test_pred = best_model.predict(test)

# 10. submission 파일에 채워 넣기
submission["BodyFat"] = test_pred
submission.to_csv("submission_knn.csv", index=False)
print("완료! submission_knn.csv 파일이 저장되었습니다.")


best params: {'knn__n_neighbors': 3, 'knn__p': 2, 'knn__weights': 'distance'}
cv best score (R^2): 0.7779551070959971
validation R^2: 0.8426459405535078
완료! submission_knn.csv 파일이 저장되었습니다.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

# Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

X = train.drop(columns=['BodyFat'])
y = train['BodyFat']

# validation split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11, 15],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2]
}

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1
)
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("CV Score:", grid.best_score_)

valid_pred = grid.predict(X_valid)
print("Validation R2:", r2_score(y_valid, valid_pred))

# Final model
best_model = grid.best_estimator_
best_model.fit(X, y)

# Predict
test_pred = best_model.predict(test)
submission["BodyFat"] = test_pred
submission.to_csv("submission_knn.csv", index=False)
print("완성: submission_knn.csv")


Best Params: {'knn__n_neighbors': 3, 'knn__p': 2, 'knn__weights': 'distance'}
CV Score: 0.7779551070959971
Validation R2: 0.8426459405535078
완성: submission_knn.csv


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor

# -------------------------
# 1. 데이터 불러오기
# -------------------------
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("submission.csv")

X = train.drop(columns=['BodyFat'])
y = train['BodyFat']

# 검증용 split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------------
# 2. 모델 파이프라인
# -------------------------
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

# -------------------------
# 3. 적당한 하이퍼파라미터로 학습
#    (너가 찾은 best params 넣으면 됨)
# -------------------------
pipe.set_params(
    knn__n_neighbors=3,
    knn__weights='distance',
    knn__p=2
)

pipe.fit(X, y)   # 전체 train으로 다시 학습

# -------------------------
# 4. test 데이터 예측
# -------------------------
test_pred = pipe.predict(test)


In [8]:
df = pd.read_csv("submission.csv")

df.dropna(axis=1, inplace=True)

# 여기!!! 이 부분을 네 예측값으로 대체!
df["BodyFat"] = test_pred

df.to_csv("new_submission.csv", index=False)
